In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("uniq_artworks_htmlLists.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(32882, 29)

In [4]:
df.head()

,_id,acquisition_year,acquisition_mode,date_creation,type,domain,domain_leaf,domain_deno_for_grid,domain_description_mst,collection,...,recap_description,all_realized_operations_history,hanging_history,hanging_history_m34,temporary_exhibitions_m30,exposition_out_of_folder,expositions,expositions_without_current,inscriptions,authors
0,150000000060949,2009.0,Don,1946,individual,Objet/Design,Presse agrumes,Objet/Design,"Presse agrumes|Aluminium, acier chromé et laqu...",NaN,...,NaN,<ul><li>2010/07/27 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,150000000000887
1,150000000078314,2002.0,Don,1968 - 1975,separable,Architecture,Dessin d'architecture,Architecture,Ensemble de dessins à inventorier,NaN,...,Dessins|Projet réalisé|1968-1975,<ul><li>2003/10/14 - 213I - Décrochage avec dé...,<ul><li>2003/07/16 - 1N4S34CO : Cimaise Ouest ...,<ul><li>Présentation des collections permanent...,NaN,NaN,NaN,<ul><li>Présentation des collections permanent...,,9000000000082144
2,150000000020652,1984.0,Achat,1980,individual,Photographie,Photographie,Photographie,Epreuve gélatino-argentique,NaN,...,NaN,<ul><li>2014/07/15 - 730I - Localisation inter...,<ul><li>1986/04/30 - CE+3S : 3ème Sud|Opératio...,NaN,NaN,NaN,"<ul><li>Masques : Arles (France), Les rencontr...","<ul><li>Masques : Arles (France), Les rencontr...",S.D.AU R. : Joel Peter Witkin/Arms broken by ....,9000000000067959
3,150000000077148,2001.0,Don,vers novembre 1966,separable,Architecture,Dessin d'architecture,Architecture,Perspective|Mine de plomb et crayon jaune sur ...,NaN,...,Projet réalisé|1963-1984,<ul><li>2008/09/23 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,9000000000081963
4,150000000476688,2011.0,Don,mars 1994 - juillet 1994,separable,Architecture,Plan,Architecture,"Dessin d'architecture, Plan|Encre de Chine, gr...",NaN,...,Projet réalisé|1992-1995,<ul><li>2011/05/10 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,150000000476606


# artist_artwork_datecreation table

In [107]:
import re
years_re = re.compile(r"\d{4}")

In [112]:
def clean_creation_date(creation_date):
    years = years_re.findall(str(creation_date))
    if len(years)>=1:
        return int(min(years))
    else:
        return None

In [114]:
df["date_creation"] = df["date_creation"].apply(clean_creation_date)

In [123]:
df_after_1990 = df.loc[df.date_creation > 1989][['_id', 'title_notice', 'authors_list', 'date_creation']]

In [124]:
df_after_1990.to_csv("artwork_created_after1990.csv", index=False)

## remove html tags

In [6]:
import re

def get_list_from_html(field):
    if(pd.isnull(field)):
        return []
    else:
        regex_item = re.compile('<li>(.*?)<\/li>', flags=re.S)
        tab = regex_item.split(str(field))
        new_tab = []
        for item in tab:
            if len(item) > 0 and 'ul>' not in item and item != '\n':
                new_tab.append(item)
        return new_tab

In [7]:
df["operation_list"] = df["all_realized_operations_history"].apply(get_list_from_html)

## remove no code cases from operation list

In [8]:
get_cleaned_list = lambda x : [i for i in x if len(i.split('-')) > 2]
remove_invalid_date = lambda x : [i for i in x if (int(i.split('-')[0][:4])>1678 and int(i.split('-')[0][:4])<2018)]
df["operation_list_clean"] = df["operation_list"].apply(get_cleaned_list).apply(remove_invalid_date)

In [9]:
# get_operation_between= lambda x : [i for i in x if (int(i.split('-')[0][:4])>1997 and int(i.split('-')[0][:4])<2001)]
# df["operation_list_1997_2000"] = df["operation_list_clean"].apply(get_operation_between)

## filter operation after 2001 && with code_loc only

In [10]:
code_loc = ["210I","211I","216I","220I", "221I","241I","260I","261I","262I","299I","730I","750I","760I"]

In [74]:
filter_operation = lambda x : [i for i in x if (int(i.split('-')[0][:4])>2000 and i.split(' - ')[1] in code_loc)]
df["operation_list_after_2001"] = df["operation_list_clean"].apply(filter_operation)

# select column

In [24]:
df_freq = df[["_id", "title_notice","domain", "authors_list", "date_creation", "operation_list_after_2001"]]

In [25]:
df_freq.shape

(32882, 6)

In [26]:
df_freq["operation_times"] = df_freq["operation_list_after_2001"].apply(lambda x: len(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df_freq = df_freq[df_freq.operation_times != 0]

In [28]:
df_freq.shape

(30767, 7)

In [35]:
sum(df_freq["operation_times"])

130519

## get opt field(date, code, opt_name, details), store into dict

In [29]:
def format_date(date):
    return date.replace("/00", "/01")

* Branche stockage : 
2AR
2B
2C
2DE
2F
2L
2M
2N
2P
2S
2V
2TR
* Branche ateliers : 
2EMB
2ENC
* Branche restauration : 
2R
2NR
2QUA
* Branche photo : 
2PHO
* Branche constats : 
2ALV 
2NALV
2CON

* Subbranch type storage : 
1B
1D
1E
1F
1I
1J
1K
1L
1M
1N3
1T
* Subbranch type restoration :
1LTR
* Subbranch type photography / digitalization : 
1STUPHO
1SAV
1STUVID
* Subbranch type studios : 
1A


In [15]:
branches = {"outside_stockage": ["2AR", "2B", "2C", "2DE", "2F", "2L", "2M", "2N", "2P", "2S", "2V", "2TR"],\
"outside_ateliers":["2EMB", "2ENC"],\
"outside_restauration":["2R", "2NR", "2QUA"],\
"outside_photo": ["2PHO"],\
"outside_constats": ["2ALV", "2NALV", "2CON"],\
"inside_stockage":["1B", "1D", "1E", "1F", "1I", "1J", "1K", "1L", "1M", "1N3", "1T"],\
"inside_restoration":["1LTR"],\
"inside_photo":["1STUPHO", "1SAV", "1STUVID"],\
"inside_studio":["1A"]}

In [16]:
def dict_mapper(pattern, dict_branch = branches):
    cand = []
    for cat, pat in dict_branch.items():
        if pattern in [pat_part[0:len(pattern)] for pat_part in pat]:
            cand += [cat]
    return cand

def get_branch(text, dict_branch = branches):
    branch_list = dict_mapper(text[0], dict_branch = dict_branch)
    if branch_list == []:
        return 'unknown'
    i = 1
    while(len(branch_list) == 1 ):
        i += 1
        branch_list = dict_mapper(text[0:i],dict_branch = dict_branch)
    return branch_list[0]

In [73]:
dict_mapper('2C')

['outside_stockage', 'outside_constats']

In [70]:
def print_branch_field(opts):
    for item in opts:
        if(item["opt_branch"] =='unknown'):
            print(item["opt_detail"])

# df_freq["operation_list_after_2001"].apply(get_movement_field)

In [67]:
get_opt_field = lambda x : [{"date": format_date(i.split(' - ')[0]), "opt_code":i.split(" - ")[1],\
                             "opt_name": i.split(" - ")[2], "opt_branch": get_branch(i.split(" - ")[3]),\
                            "opt_detail": "-".join(i.split(" - ")[3:])}\
                            for i in x]

In [68]:
df_freq["opt_field"] = df_freq["operation_list_after_2001"].apply(get_opt_field)

In [71]:
df_freq["opt_field"].apply(print_branch_field)

DONATEUR-Dossier n° 9085-E10
6BBRV-Dossier n° 5402-E10
ECT-Dossier n° 4088-M32-Parade 1901-2001 Collections du Centre Pompidou Musée national d'art moderne Centre de création industrielle
DONATEUR-Dossier n° 9085-E10
ECT (pour installation au N4, G.Herbaux)
ECT-Dossier n° 13037-E10
6BBRV-Dossier n° 5714-E10
6BRAR
6BBRV-Dossier n° 5714-E10
9 (Unité de désinfection de Bussy St Georges)-Dossier n° 7580-M20-Poétique du Chantier
DONATEUR-Dossier n° 9085-E10
DONATEUR-Dossier n° 9085-E10
ECT-Dossier n° 9799-M34-Une histoire – art, architecture et design des années 1980 à nos jours
DONATEUR-Dossier n° 9085-E10
6BRAR-Dossier n° 5665-I90-Rangement dessins documentaires Pol Abraham
 -Dossier n° 9044-M34-Modernités plurielles 1905-1970, présentation des collections permanentes
6BBRV-Dossier n° 5088-I03-VI : fonds sonore John Giorno
DONATEUR-Dossier n° 9085-E10
 -Dossier n° 9044-M34-Modernités plurielles 1905-1970, présentation des collections permanentes
ECT-Dossier n° 6242-I20-Décrochage Big-Bang

ECT (transfert à PN pour restauration)-Dossier n° 8478-M20-Henri Matisse et les Fauves
DONATEUR-Dossier n° 9085-E10
ECT-Dossier n° 13254-M34-Focus Grigorio Gregotti
ECT-Dossier n° 4405-I20-Niveau 5
ECT-Dossier n° 4986-I20-Niveau 4
ECT (transfert PN > CP hommage à Pompidou)-Dossier n° 6601-I20-Accrochage NIV 4-avril 2007
ECT (MRT2)-Dossier n° 6116-I10-Transfert Centre-Paris Nord
ECT-Dossier n° 6302-I20-accrochage Donateurs
ECT-Dossier n° 11826-M20-Chagall et la Musique// Chagall: couleur et musique
6BBRV-Dossier n° 5714-E10
6BBRV-Dossier n° 6053-M20-Collection Nouveaux Médias
6BBRV-Dossier n° 4740-I10-PPRI : CENTRE > Réserve VI (nouveaux medias)
ECT-Dossier n° 8120-M36-Vues d'en haut
ECT-Dossier n° 9044-M34-Modernités plurielles 1905-1970, présentation des collections permanentes
ECT-Dossier n° 7519-M34-Elles@centrepompidou
6BBRV-Dossier n° 6053-M20-Collection Nouveaux Médias
6BBRV-Dossier n° 5449-M32-Modernité / Modernitet : oeuvres suédoises de la collection du Centre Pompidou. De Vik

0        None
1        None
2        None
3        None
4        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
31       None
32       None
         ... 
32849    None
32850    None
32851    None
32852    None
32853    None
32854    None
32855    None
32856    None
32858    None
32859    None
32860    None
32861    None
32862    None
32863    None
32864    None
32865    None
32866    None
32867    None
32868    None
32869    None
32870    None
32871    None
32873    None
32874    None
32875    None
32876    None
32878    None
32879    None
32880    None
32881    None
Name: opt_field, Length: 30767, dtype: object

In [75]:
df_freq.drop(["operation_times"], axis=1)

,_id,title_notice,domain,authors_list,date_creation,operation_list_after_2001,opt_field
0,150000000060949,Juice King JK40,Objet/Design,NATIONAL DIE CASTING,1946,[2010/07/27 - 210I - Déplacement vers loc. int...,"[{'date': '2010/07/27', 'opt_code': '210I', 'o..."
1,150000000078314,Ensemble de dessins,Architecture,DOTTELONDE René,1968 - 1975,[2003/10/14 - 213I - Décrochage avec déplaceme...,"[{'date': '2003/10/14', 'opt_code': '213I', 'o..."
2,150000000020652,Arms broken by windows|(Bras cassés par la fen...,Photographie,WITKIN Joel-Peter,1980,[2014/07/15 - 730I - Localisation interne conf...,"[{'date': '2014/07/15', 'opt_code': '730I', 'o..."
3,150000000077148,Croquis d'implantation,Architecture,BALLADUR Jean,vers novembre 1966,[2008/09/23 - 210I - Déplacement vers loc. int...,"[{'date': '2008/09/23', 'opt_code': '210I', 'o..."
4,150000000476688,"Plan étage inférieur, feuille 3",Architecture,ACAYABA Marcos,mars 1994 - juillet 1994,[2011/05/10 - 210I - Déplacement vers loc. int...,"[{'date': '2011/05/10', 'opt_code': '210I', 'o..."
5,150000000023357,Madame Dufy,Dessin,DUFY Raoul,[1953],[2005/01/24 - 730E - Localisation confirmée - ...,"[{'date': '2005/01/24', 'opt_code': '730E', 'o..."
6,150000000071485,Mlle Barcinska : l'accordéoniste,Photographie,"MAN RAY (RADNITZKY Emmanuel, dit)",1932,[2006/04/07 - 211I - Saisie localisation inter...,"[{'date': '2006/04/07', 'opt_code': '211I', 'o..."
7,150000000046930,Bubu 1er,Objet/Design,STARCK Philippe,1996,[2008/11/19 - 210I - Déplacement vers loc. int...,"[{'date': '2008/11/19', 'opt_code': '210I', 'o..."
8,150000000053878,Lee Miller,Photographie,"MAN RAY (RADNITZKY Emmanuel, dit)",vers 1930,[2006/04/07 - 211I - Saisie localisation inter...,"[{'date': '2006/04/07', 'opt_code': '211I', 'o..."
9,150000000040273,Autoportrait nu avec un buste en plâtre,Photographie,"MAN RAY (RADNITZKY Emmanuel, dit)",vers 1930,[2006/04/07 - 211I - Saisie localisation inter...,"[{'date': '2006/04/07', 'opt_code': '211I', 'o..."


In [76]:
df_freq.to_json("artwork_operations_2001.json", orient="records")